<a href="https://colab.research.google.com/github/chetools/CHE4071_Fall2025/blob/main/dynamic_distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.1 MB/s eta 0:00:00


In [2]:
from importnb import Notebook
with Notebook():
    from che5 import sim, pid, TF1, TF2, shift

import numpy as np
import scipy as sp
import scipy.signal as sig
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_dark"

import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

import sympy
from sympy.abc import s
from sympy import exp, Symbol, simplify


In [40]:
N=13
Nfeed = 7
alpha = 2
F = 1.  #mol/s feed flow rate
z = 0.5 #mol fraction of more volatile component in feed
q = 0.9  #liquid mole fraction of feed
Dsp = 0.5 * F  #fraction of feed desired as distillate flow
R = 2. #reflux ratio

Vstrip = Dsp + R*Dsp -  F*(1-q)  # vapor flow in stripping section
Vrec = Vstrip +   F*(1-q)  #vapor in rectifying section
m0tray = 0.2 #tray holdup in moles at zero flow
m0cond = 0.5 #condenser holdup at no flow
m0boil = 0.5 #boiler holdup at no flow
m0 = np.r_[m0cond, (m0tray,)*N, m0boil]
m_ic = np.full(N+2,10.)  #fill up condenser, reboiler, and trays with 0.8 moles of liquid
x_ic = np.full(N+2,z) #fill up all stages with feed composition
V = np.r_[0., (Vrec,)*Nfeed, (Vstrip,)*(N-Nfeed+1)]
weir_const = 0.1 #lumped Weir constant that includes effect of weir width, density, viscoisty, conversion from volumetric to molar flows

In [41]:
def rhs_m_only(t, vec):
    m = vec
    mdiff = m-m0
    L=weir_const*(np.where(mdiff<0, 0., mdiff))**(1.5)
    D = L[0]/(R+1)
    L[0] = R*D
    dm=np.zeros(N+2)
    dm-=L
    dm[0]-=D
    dm[1:]+=L[:-1]
    dm[Nfeed]+=F
    dm[:-1]+=V[1:]
    dm[1:]-=V[1:]
    return dm

In [42]:
tend=100
res=sp.integrate.solve_ivp(rhs_m_only, (0,tend), m_ic, method='Radau', dense_output=True)
tplot = np.linspace(0,tend,200)
m_sims = res.sol(tplot)
fig = make_subplots()
for i,m_sim in enumerate(m_sims):
    fig.add_scatter(x=tplot, y=m_sim, name=f'{i}')
fig.update_layout(width=800,height=500)

In [43]:
#alpha = K1/K2 = y1*x2 / (x1*y2) = y1*(1-x1) / (x1*(1-y1))
#alpha*x  =   y- y*x + alpha*x*y = y*(1 - x + alpha*x)
# y = alpha*x/ (1 - x + alpha*x)

def rhs(t, vec):
    x, m = np.split(vec,2)
    y = alpha*x/ (1 - x + alpha*x)
    mdiff = m-m0
    L=weir_const*(np.where(mdiff<0, 0., mdiff))**(1.5)
    D = L[0]/(R+1)
    L[0] = R*D
    dm=np.zeros(N+2)
    dm-=L
    dm[0]-=D
    dm[1:]+=L[:-1]
    dm[Nfeed]+=F
    dm[:-1]+=V[1:]
    dm[1:]-=V[1:]

    #m is number of moles, x is mole fraction of more volatile component
    #m*x is number of moles of more volatile component at each stage (condenser, tray, or reboiler)
    dmx=np.zeros(N+2)
    dmx-=L*x
    dmx[0]-=D*x[0]
    dmx[1:]+=L[:-1]*x[:-1]
    dmx[Nfeed]+=F*z
    dmx[:-1]+= V[1:] * y[1:]
    dmx[1:]-= V[1:] * y[1:]

    dx = (dmx - x*dm)/m
    return np.r_[dx, dm]

In [56]:
tend=200
res=sp.integrate.solve_ivp(rhs, (0,tend), np.r_[x_ic, m_ic], method='Radau', dense_output=True)

In [57]:
tplot = np.linspace(0,tend,200)
x_sims, m_sims = np.split(res.sol(tplot),2,axis=0)
fig = make_subplots(rows=1,cols=2)
for i,m_sim in enumerate(m_sims):
    fig.add_scatter(x=tplot, y=m_sim, name=f'm{i}',row=1,col=1)
for i,x_sim in enumerate(x_sims):
    fig.add_scatter(x=tplot, y=x_sim, name=f'x{i}',row=1,col=2)
fig.update_layout(width=1000,height=500)

In [58]:
xm_ss1 = res.y[:,-1]
xs, ms = np.split(xm_ss1,2)
ys = alpha*xs/ (1 - xs + alpha*xs)
xmc = np.r_[xs[0], np.repeat(xs[1:-1],2), xs[-1]]
ymc = np.repeat(ys[1:],2)
xmc, ymc

(array([0.91912334, 0.85051254, 0.85051254, 0.77551743, 0.77551743,
        0.70016563, 0.70016563, 0.63055886, 0.63055886, 0.5710419 ,
        0.5710419 , 0.52338801, 0.52338801, 0.4871623 , 0.4871623 ,
        0.46167056, 0.46167056, 0.4248688 , 0.4248688 , 0.37483079,
        0.37483079, 0.31220394, 0.31220394, 0.24153709, 0.24153709,
        0.1705769 , 0.1705769 , 0.10722005]),
 array([0.91921835, 0.91921835, 0.8735678 , 0.8735678 , 0.82364402,
        0.82364402, 0.77342668, 0.77342668, 0.72695948, 0.72695948,
        0.68713684, 0.68713684, 0.65515687, 0.65515687, 0.63170262,
        0.63170262, 0.59636199, 0.59636199, 0.54527553, 0.54527553,
        0.47584667, 0.47584667, 0.38909363, 0.38909363, 0.29144074,
        0.29144074, 0.19367433, 0.19367433]))

In [59]:
xeq = np.linspace(0,1, 100)
yeq = alpha*xeq/ (1 - xeq + alpha*xeq)
fig = make_subplots()
fig.add_scatter(x=xeq, y=yeq, mode='lines', line_color='rgba(199, 0, 255, 0.8)')
fig.add_scatter(x=xmc, y=ymc, mode='lines')
fig.add_scatter(x=[0,1], y=[0,1], line_color='grey', mode='lines')
# fig.add_scatter(x=tplot, y=x_sim, name=f'x{i}')
fig.update_layout(width=500,height=500, showlegend=False)

In [60]:
def ramp_factory(y1, y2, x1, x2):

    def ramp(t):
        if t<x1:
            return y1
        elif t>x2:
            return y2
        else:
            return y1 + (y2-y1)/(x2-x1)*(t-x1)

    return ramp

In [61]:
R_ramp = ramp_factory(2., 4., 20, 150.)

In [62]:
tplot = np.linspace(0,500,200)
Rs = []
for t in tplot:
    Rs.append(R_ramp(t))
fig = make_subplots()
fig.add_scatter(x=tplot, y=Rs, mode='lines', line_color='rgba(199, 0, 255, 0.8)')
fig.update_layout(width=300,height=300, showlegend=False)

In [63]:
def rhs_dynamicR(t, vec):
    x, m = np.split(vec,2)
    y = alpha*x/ (1 - x + alpha*x)

    R = R_ramp(t) #reflux ratio is ramped.
    Vstrip = Dsp + R*Dsp -  F*(1-q)  # vapor flow in stripping section
    Vrec = Vstrip +   F*(1-q)  #vapor in rectifying section
    V = np.r_[0., (Vrec,)*Nfeed, (Vstrip,)*(N-Nfeed+1)]

    mdiff = m-m0
    L=weir_const*(np.where(mdiff<0, 0., mdiff))**(1.5)
    D = L[0]/(R+1)
    L[0] = R*D
    dm=np.zeros(N+2)
    dm-=L
    dm[0]-=D
    dm[1:]+=L[:-1]
    dm[Nfeed]+=F
    dm[:-1]+=V[1:]
    dm[1:]-=V[1:]

    #m is number of moles, x is mole fraction of more volatile component
    #m*x is number of moles of more volatile component at each stage (condenser, tray, or reboiler)
    dmx=np.zeros(N+2)
    dmx-=L*x
    dmx[0]-=D*x[0]
    dmx[1:]+=L[:-1]*x[:-1]
    dmx[Nfeed]+=F*z
    dmx[:-1]+= V[1:] * y[1:]
    dmx[1:]-= V[1:] * y[1:]

    dx = (dmx - x*dm)/m
    return np.r_[dx, dm]

In [64]:
res=sp.integrate.solve_ivp(rhs_dynamicR, (0,tend), xm_ss1, method='Radau', dense_output=True)

In [65]:
tsol = np.linspace(0,tend,6)

In [66]:
xs_ts, ms_ts = np.split(res.sol(tsol),2,axis=0)

In [67]:
xeq = np.linspace(0,1, 100)
yeq = alpha*xeq/ (1 - xeq + alpha*xeq)
fig = make_subplots(rows=1,cols=xs_ts.shape[1])
for i in range(xs_ts.shape[1]):
    xs = xs_ts[:,i]
    ys = alpha*xs/ (1 - xs + alpha*xs)
    xmc = np.r_[xs[0], np.repeat(xs[1:-1],2), xs[-1]]
    ymc = np.repeat(ys[1:],2)
    fig.add_scatter(x=xeq, y=yeq, mode='lines', line_color='rgba(199, 0, 255, 0.8)', row=1,col=i+1)
    fig.add_scatter(x=xmc, y=ymc, mode='lines', row=1, col=i+1)
    fig.add_scatter(x=[0,1], y=[0,1], line_color='grey', mode='lines', row=1, col=i+1)
fig.update_layout(width=1200, height=400, template='plotly_dark',showlegend=False)

IndexError: tuple index out of range

In [22]:


fig.add_scatter(x=xeq, y=yeq, mode='lines', line_color='rgba(199, 0, 255, 0.8)')
fig.add_scatter(x=xmc, y=ymc, mode='lines')
fig.add_scatter(x=[0,1], y=[0,1], line_color='grey', mode='lines')
# fig.add_scatter(x=tplot, y=x_sim, name=f'x{i}')
fig.update_layout(width=500,height=500, showlegend=False)